### this allows await to run in cells  instead of  async fct

In [3]:
# %pip install --upgrade ipykernel ipython

In [4]:
SEARCH_KEYWORDS = "Site Reliability Engineer"
LOCATION = "Canada"

JOB_SEARCH_PAGE_URL = "https://www.linkedin.com/jobs/search"
JOB_SEARCH_API_URL = "https://www.linkedin.com/jobs-guest/jobs/api/seeMoreJobPostings/search"
import asyncio
from curl_cffi.requests import AsyncSession
from bs4 import BeautifulSoup, Tag


# this fction extracts the job info from the html card

In [5]:

def parse_job_card(job_card: Tag):
    href_tag = job_card.find("a", class_="base-card__full-link")
    job_url = href_tag["href"].split("?")[0] if href_tag else None
    job_id = job_url.split("-")[-1] if job_url else None

    title_tag = job_card.find("span", class_="sr-only")
    title = title_tag.get_text(strip=True) if title_tag else None

    company_tag = job_card.find("h4", class_="base-search-card__subtitle")
    company = company_tag.get_text(strip=True) if company_tag else None

    location_tag = job_card.find("span", class_="job-search-card__location")
    location = location_tag.get_text(strip=True) if location_tag else None

    datetime_tag = job_card.find("time", class_="job-search-card__listdate")
    date_posted = datetime_tag["datetime"] if datetime_tag and "datetime" in datetime_tag.attrs else None

    return {
        "job_id": job_id,
        "title": title,
        "company": company,
        "location": location,
        "date_posted": date_posted,
        "job_url": job_url,
    }


### initialize the session

In [6]:
session = AsyncSession(impersonate="chrome120")

### we hit the main page first to get the cookies

In [7]:
prime_params = {'keywords': SEARCH_KEYWORDS, 'location': LOCATION}
await session.get(JOB_SEARCH_PAGE_URL, params=prime_params)
api_params=prime_params



# first request to get the first batch of jobs

In [13]:
api_params.update({'start':900000})
api_response = await session.get(JOB_SEARCH_API_URL, params=api_params)


soup = BeautifulSoup(api_response.text, 'html.parser')
job_cards = soup.find_all("div", class_="base-search-card")

all_jobs = [parse_job_card(card) for card in job_cards]


In [ ]:
start=0
job_list=[]

while 1:
    api_response = await session.get(JOB_SEARCH_API_URL, params=api_params)
    soup = BeautifulSoup(api_response.text, 'html.parser')
    job_cards = soup.find_all("div", class_="base-search-card")

    all_jobs = [parse_job_card(card) for card in job_cards]
    job_list.append(all_jobs)



    if api_response.status_code==400: break
    else: start+=25

